In [ ]:
import Pkg
Pkg.add("DataFrames")
Pkg.add("DataFramesMeta")
Pkg.add("Combinatorics")


In [8]:
using JuMP, GLPK, LinearAlgebra

c = [-1.1; -1; 0; 0; 0] # coefficients of objective function
A = [5/3 1 1 0 0;
     8/7 1 0 1 0;
     3/7 1 0 0 1] # coefficients of constraints function in standard form
b = [5; 4; 3] # RHS of constraints

m,n = size(A) # m is the no of constraints and n is the number of variables
x1_b = [0; 0; 0; 0; 0] # x[j=1:n] >= x1_b[j=1:n]

# set up problem
myModel = Model(with_optimizer(GLPK.Optimizer))
@variable(myModel, x[j=1:n] >=x1_b[j]) # assign j value in x[] (create x_1 to x_n) and use j value in x1_b[].
for i = 1:m   
    @constraint(myModel, sum(A[i,j] *x[j] for j=1:n) == b[i])
end
@objective(myModel, Min, sum(c[j]*x[j] for j=1:n))

print(myModel)

# Question: @variable(myModel, x[j] >=x1_b[j] for j=1:n) does not work?
# @objective(myModel, Min, sum(c[j=1:n]*x[j])) does not work?


Min -1.1 x[1] - x[2]
Subject to
 1.6666666666666667 x[1] + x[2] + x[3] = 5.0
 1.1428571428571428 x[1] + x[2] + x[4] = 4.0
 0.42857142857142855 x[1] + x[2] + x[5] = 3.0
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[4] ≥ 0.0
 x[5] ≥ 0.0


In [28]:
# Search Method
using DataFrames, DataFramesMeta, Combinatorics
# Find all basic solutions
combs = collect(combinations(1:n, m)) # all combinations of m variables from n variables
result = DataFrame(comb_1=NaN, comb_2=NaN, comb_3=NaN, x_B_1=NaN, x_B_2=NaN, x_B_3=NaN, z=NaN) # Store results

for i in 1:length(combs)
    comb = combs[i]
    B = A[:, comb]
    c_B = c[comb]
    x_B = inv(B)*b # Find value for each x
# Check feasibility of basic solution: If feasible, calculate utility  
    if minimum(x_B) > 0 
        z = dot(c_B,x_B)
    else
        z = Inf
    end
    
    if i==1
        result[1,:] = [comb[1], comb[2], comb[3], x_B[1], x_B[2], x_B[3], z] # This is to replace the original row of NaN
    else
        push!(result, [comb[1], comb[2], comb[3], x_B[1], x_B[2], x_B[3], z]) # Insert new row to the DF at the end of last row
    end
end
# Sort rows
sort(result, :z)


# Among the 10 basic solutions, there are 5 feasible basic solutions and one optimal feasible basic solution
# The optimal basic solution is when x=[1.4; 2.4; 0.2667; 0; 0] and has utility of -3.94
# My remaining question is why choose m among n at the very beginning?? 

,comb_1,comb_2,comb_3,x_B_1,x_B_2,x_B_3,z
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,2.0,3.0,1.4,2.4,0.266667,-3.94
2,1.0,2.0,5.0,1.90909,1.81818,0.363636,-3.91818
3,1.0,4.0,5.0,3.0,0.571429,1.71429,-3.3
4,2.0,3.0,4.0,3.0,2.0,1.0,-3.0
5,3.0,4.0,5.0,5.0,4.0,3.0,0.0
6,1.0,2.0,4.0,1.61538,2.30769,-0.153846,Inf
7,1.0,3.0,4.0,7.0,-6.66667,-4.0,Inf
8,1.0,3.0,5.0,3.5,-0.833333,1.5,Inf
9,2.0,3.0,5.0,4.0,1.0,-1.0,Inf


In [ ]:
# Simplex Method
using JuMP, GLPK
# Set up model 
newModel = Model(with_optimizer(GLPK.Optimizer))


@variable(newModel, x[i=1:n] >=x_lb[i])
for i=1:m
    @constraint(newModel, sum(A[i,j]*x[j] for j=1:n) == b[i])
end
    @objective(newModel, Min, sum(c[j]*x[j] for j=1:n))
println("The optimization problem to be solved is:")
print(newModel)
println(" ")
println("The rank of the matrix A: ",rank(A))
println("The number of linear restrictions: ", m)
println("The number of variables: ",n)
println("Number of basic solutions n!/m!(n-m)!: ",factorial(n)/(factorial(m)*factorial(n-m)))